# HiddenGuard: Colab Notebook
Use this notebook to run the **HiddenGuard** pipeline for detecting steganographic prompt injection (SPI) against MLLM pipelines.

**Options to load the repo:**

- **A) GitHub clone** – set `REPO_URL` below (recommended after you upload your repo to GitHub)

- **B) Upload ZIP** – upload the ZIP you downloaded from ChatGPT and extract it here



## 0. Runtime checks

In [ ]:

import sys, platform, torch, subprocess, os, json
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU runtime")


## 1. Get the code
Choose **one** of the options below.

### Option A — Clone from GitHub (edit the URL)

In [ ]:

# TODO: Replace with your GitHub URL once you push the repo
REPO_URL = "https://github.com/<your-org-or-user>/HiddenGuard.git"

if REPO_URL.startswith("https://github.com/<"):
    print("⚠️ Please set REPO_URL to your GitHub repository URL (then re-run this cell).")
else:
    !rm -rf HiddenGuard
    !git clone --depth=1 "$REPO_URL" HiddenGuard
    %cd HiddenGuard
    !ls -la


### Option B — Upload the ZIP and extract

In [ ]:

# Run this if you want to upload the ZIP created by ChatGPT.
# After running, click the folder icon (left bar) to verify files extracted under ./HiddenGuard

import shutil, os, zipfile
from google.colab import files

upload = files.upload()
zip_name = list(upload.keys())[0]
print("Uploaded:", zip_name)

# Extract to ./HiddenGuard
!rm -rf HiddenGuard
os.makedirs("HiddenGuard", exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall(".")

# Some zips include the parent folder; auto-detect
if os.path.isdir("mnt/data/HiddenGuard"):
    # If the zip preserved its original path structure, move it into ./HiddenGuard
    shutil.move("mnt/data/HiddenGuard", "./HiddenGuard")
    # Clean parent dirs if created by extraction
    if os.path.isdir("mnt"):
        shutil.rmtree("mnt")

# Ensure we are inside the project root
if os.path.isdir("HiddenGuard"):
    %cd HiddenGuard
    !ls -la
else:
    print("❗Could not locate HiddenGuard folder. Please explore the file tree and cd into the correct directory.")


## 2. Install dependencies

In [ ]:

# Colab may need these system libs
!apt-get update -qq && apt-get install -y -qq libgl1

# Python dependencies
!pip install -q --upgrade pip
!pip install -q -r requirements.txt

# (Optional) If you see CLIP download warnings, they will resolve on first use.


## 3. Sanity check project structure

In [ ]:

import os, json, pathlib
tree = []
for root, dirs, files in os.walk(".", topdown=True):
    if any(x.startswith(".venv") for x in dirs): 
        dirs[:] = [d for d in dirs if not d.startswith(".venv")]
    if root.count(os.sep) > 3:
        continue
    for f in files:
        if f.endswith((".py",".md",".yaml",".yml")):
            tree.append(os.path.join(root, f))
print("\n".join(sorted(tree[:200])))


## 4. Run quick scans with the CLI

In [ ]:

!python scripts/scan_image.py --image data/samples/safe_noise.png --text "A noisy abstract pattern"
!python scripts/scan_image.py --image data/samples/stego_message.png --text "A noisy abstract pattern"


## 5. Launch the REST API (background)

In [ ]:

# Start FastAPI in the background (Colab cell). You can stop it by interrupting the cell.
import subprocess, sys, time
server = subprocess.Popen([sys.executable, "-m", "uvicorn", "src.hidden_guard.api.app:app", "--host", "0.0.0.0", "--port", "8000"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
time.sleep(3)
print("API should be running on port 8000. Use the next cell to query it.")


### Test the API endpoint

In [ ]:

import requests
from pathlib import Path

img_path = Path("data/samples/stego_message.png")
files = {"image": ("stego_message.png", open(img_path, "rb"), "image/png")}
data = {"text": "A noisy abstract pattern"}
r = requests.post("http://127.0.0.1:8000/scan", files=files, data=data, timeout=60)
print(r.status_code)
print(r.json())


## 6. Latency benchmark (optional)

In [ ]:

!python scripts/bench_latency.py --image data/samples/stego_message.png --text "abstract pattern" --iters 30


## 7. Ablation study (component-wise outputs)

In [ ]:

!python -m src.hidden_guard.eval.ablation --image data/samples/stego_message.png --text "abstract pattern"


## 8. Visualize wavelet-frequency cues

In [ ]:

# Display the image and compute WSD internal features for inspection
import numpy as np, cv2, json
from src.hidden_guard.wsd import WaveletStegoDetector

wsd = WaveletStegoDetector()
score, feats = wsd.score("data/samples/stego_message.png")
print("WSD suspicious score:", score)
print("Top features:", {k: round(v, 4) for k, v in list(feats.items())[:5]})

# Simple histogram of pixel intensities as a proxy (no fancy style/colors)
import matplotlib.pyplot as plt
img = cv2.imread("data/samples/stego_message.png", cv2.IMREAD_GRAYSCALE)
plt.figure()
plt.hist(img.ravel(), bins=32)
plt.title("Intensity Histogram (Stego Sample)")
plt.xlabel("Gray Level")
plt.ylabel("Count")
plt.show()


## 9. Stop the API server

In [ ]:

try:
    server.terminate()
    server.wait(timeout=3)
    print("Server stopped.")
except Exception as e:
    print("Server may already be stopped.", e)
